# Piecewise Conversion

Model equipment with load-dependent efficiency using piecewise linear approximations.

This notebook covers:

- **PiecewiseConversion**: Non-linear input/output relationships
- **Part-load efficiency**: Equipment performance varies with operating point
- **Time-varying piecewise**: Boundaries that change over time (e.g., temperature-dependent)
- **Model refinement**: Comparing simple vs detailed efficiency models

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

## The Problem: Load-Dependent Efficiency

Real equipment efficiency varies with load level:

- **Part load** (30-50%): Lower efficiency due to throttling losses
- **Mid load** (50-80%): Optimal efficiency at design point
- **Full load** (80-100%): Slightly reduced efficiency

This non-linear relationship can be approximated using **piecewise linear segments**.

### Example: Gas Engine Efficiency

| Load Range | Electrical Efficiency |
|------------|----------------------|
| 30-50% | 32-38% |
| 50-75% | 38-42% |
| 75-100% | 42-40% |

### When to Use This Approach

Use `PiecewiseConversion` when:
- Efficiency depends on the **operating load level**
- You have **efficiency curve data** from manufacturer specs
- Part-load operation is **significant** in your scenario

## Define the Efficiency Curve

We define corresponding segments for fuel input and electrical output:

In [ ]:
# Gas engine efficiency varies with load:
# - Part load: lower efficiency (~32-38%)
# - Mid load: optimal efficiency (~38-42%)
# - High load: slightly lower efficiency (~42-40%)

SIMPLE_EFFICIENCY = 0.38  # Average efficiency for comparison

# Piecewise efficiency model
# Each segment defines corresponding input/output ranges
piecewise_efficiency = fx.PiecewiseConversion(
    {
        'Fuel': fx.Piecewise(
            [
                fx.Piece(start=78, end=132),  # Part load: 78-132 kW fuel
                fx.Piece(start=132, end=179),  # Mid load: 132-179 kW fuel
                fx.Piece(start=179, end=250),  # High load: 179-250 kW fuel
            ]
        ),
        'Elec': fx.Piecewise(
            [
                fx.Piece(start=25, end=50),  # Part load: 25-50 kW elec (32-38% eff)
                fx.Piece(start=50, end=75),  # Mid load: 50-75 kW elec (38-42% eff)
                fx.Piece(start=75, end=100),  # High load: 75-100 kW elec (42-40% eff)
            ]
        ),
    }
)

### Understanding the Segments

Each `Piece` defines a linear segment between start and end points. The segments connect to form a piecewise linear curve:

- **Segment 1** (Part load): Fuel 78→132, Elec 25→50 (efficiency: 32%→38%)
- **Segment 2** (Mid load): Fuel 132→179, Elec 50→75 (efficiency: 38%→42%)
- **Segment 3** (High load): Fuel 179→250, Elec 75→100 (efficiency: 42%→40%)

## Create Demand Profile

In [ ]:
# 2-day demand profile with varying load
timesteps = pd.date_range('2024-01-22', periods=48, freq='h')

# Demand varies from 30 kW to 90 kW
elec_demand = np.concatenate(
    [
        np.linspace(30, 90, 24),  # Day 1: ramping up
        np.linspace(90, 30, 24),  # Day 2: ramping down
    ]
)

px.line(x=timesteps, y=elec_demand, title='Electricity Demand Profile', labels={'x': 'Time', 'y': 'Demand [kW]'})

## Model 1: Simple Constant Efficiency

First, let's build a baseline model with constant average efficiency.

In [ ]:
fs_simple = fx.FlowSystem(timesteps)

fs_simple.add_elements(
    # Buses
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    # Cost tracking
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    # Gas supply
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    # Gas engine with CONSTANT efficiency
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas', size=300)],
        outputs=[fx.Flow('Elec', bus='Electricity', size=100)],
        conversion_factors=[{'Fuel': 1, 'Elec': SIMPLE_EFFICIENCY}],  # 38% constant
    ),
    # Electricity demand
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand)]),
)

fs_simple.optimize(fx.solvers.HighsSolver())

## Model 2: Piecewise Load-Dependent Efficiency

Now the refined model using `PiecewiseConversion`.

In [ ]:
fs_piecewise = fx.FlowSystem(timesteps)

fs_piecewise.add_elements(
    # Buses
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    # Cost tracking
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    # Gas supply
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    # Gas engine with PIECEWISE efficiency
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas')],  # No size needed - defined by piecewise
        outputs=[fx.Flow('Elec', bus='Electricity')],
        piecewise_conversion=piecewise_efficiency,  # <-- Piecewise efficiency curve
    ),
    # Electricity demand
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand)]),
)

fs_piecewise.optimize(fx.solvers.HighsSolver())

## Visualize the Efficiency Curve

In [ ]:
# Plot the piecewise conversion curve
fs_piecewise.components['GasEngine'].piecewise_conversion.plot(
    x_flow='Fuel', title='Gas Engine: Fuel Input vs Electricity Output'
)

## Compare Results

In [ ]:
# Compare costs
simple_cost = fs_simple.solution['costs'].item()
piecewise_cost = fs_piecewise.solution['costs'].item()

print(f'Simple model (38% constant):    {simple_cost:.2f} €')
print(f'Piecewise model (load-varying): {piecewise_cost:.2f} €')
print(f'Difference: {piecewise_cost - simple_cost:.2f} € ({(piecewise_cost - simple_cost) / simple_cost * 100:.1f}%)')

In [ ]:
# Visualize the operation
fs_piecewise.statistics.plot.balance('Gas')

In [ ]:
fs_piecewise.statistics.plot.balance('Electricity')

In [ ]:
# Compare actual efficiency at each timestep using xarray for auto-alignment
simple_data = xr.Dataset(
    {
        'fuel': fs_simple.solution['GasEngine(Fuel)|flow_rate'],
        'elec': fs_simple.solution['GasEngine(Elec)|flow_rate'],
    }
)
simple_data['efficiency'] = xr.where(simple_data['fuel'] > 0.1, simple_data['elec'] / simple_data['fuel'], np.nan)

piecewise_data = xr.Dataset(
    {
        'fuel': fs_piecewise.solution['GasEngine(Fuel)|flow_rate'],
        'elec': fs_piecewise.solution['GasEngine(Elec)|flow_rate'],
    }
)
piecewise_data['efficiency'] = xr.where(
    piecewise_data['fuel'] > 0.1, piecewise_data['elec'] / piecewise_data['fuel'], np.nan
)

# Plot comparison
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=simple_data['time'].values,
        y=simple_data['efficiency'].values,
        name='Simple (constant)',
        line=dict(color='blue'),
    )
)
fig.add_trace(
    go.Scatter(
        x=piecewise_data['time'].values,
        y=piecewise_data['efficiency'].values,
        name='Piecewise (load-varying)',
        line=dict(color='red'),
    )
)
fig.update_layout(
    title='Operating Efficiency Comparison',
    xaxis_title='Time',
    yaxis_title='Efficiency',
)
fig.show()

---

## Time-Varying Piecewise Boundaries

Real equipment often has operating limits that **change over time**:

- **Ambient temperature** affects maximum capacity (e.g., gas turbines derate in hot weather)
- **Fuel quality** varies seasonally
- **Maintenance schedules** reduce available capacity

`Piece` boundaries can accept **arrays** instead of scalars to model these effects.

### Example: Temperature-Dependent Gas Engine Capacity

A gas engine's maximum output decreases when ambient temperature rises:
- At 0°C: Full capacity (100 kW electrical)
- At 30°C: Reduced to 85 kW electrical (derating)

In [ ]:
# Create a temperature profile (daily cycle)
hours = np.arange(48)
hour_of_day = hours % 24

# Temperature varies: cool at night (5°C), hot in afternoon (28°C)
temperature = 15 + 13 * np.sin((hour_of_day - 6) * np.pi / 12)
temperature = np.clip(temperature, 2, 30)

# Derating factor: 1.0 at 0°C, 0.85 at 30°C (linear)
derating = 1.0 - 0.15 * (temperature / 30)

# Visualize temperature and derating
fig = go.Figure()
fig.add_trace(go.Scatter(x=timesteps, y=temperature, name='Temperature [°C]', yaxis='y1'))
fig.add_trace(go.Scatter(x=timesteps, y=derating * 100, name='Capacity [%]', yaxis='y2'))
fig.update_layout(
    title='Ambient Temperature and Engine Derating',
    yaxis=dict(title='Temperature [°C]', side='left'),
    yaxis2=dict(title='Capacity [%]', side='right', overlaying='y', range=[80, 105]),
    legend=dict(x=0.02, y=0.98),
)
fig.show()

In [ ]:
# Define TIME-VARYING piecewise boundaries
# Maximum capacity changes with temperature

# Base values (at full capacity)
fuel_max_base = 250
elec_max_base = 100

# Apply derating to maximum values (arrays!)
fuel_max_derated = fuel_max_base * derating
elec_max_derated = elec_max_base * derating

# Scale intermediate points proportionally
piecewise_time_varying = fx.PiecewiseConversion(
    {
        'Fuel': fx.Piecewise(
            [
                fx.Piece(start=78 * derating, end=132 * derating),  # Part load (scaled)
                fx.Piece(start=132 * derating, end=179 * derating),  # Mid load (scaled)
                fx.Piece(start=179 * derating, end=fuel_max_derated),  # High load (scaled)
            ]
        ),
        'Elec': fx.Piecewise(
            [
                fx.Piece(start=25 * derating, end=50 * derating),  # Part load (scaled)
                fx.Piece(start=50 * derating, end=75 * derating),  # Mid load (scaled)
                fx.Piece(start=75 * derating, end=elec_max_derated),  # High load (scaled)
            ]
        ),
    }
)

print(f'Fuel max range: {fuel_max_derated.min():.1f} - {fuel_max_derated.max():.1f} kW')
print(f'Elec max range: {elec_max_derated.min():.1f} - {elec_max_derated.max():.1f} kW')

### Visualize Time-Varying Boundaries

In [ ]:
# Plot the time-varying boundaries for electricity output
fig = go.Figure()

# Add each segment boundary as a line
fig.add_trace(go.Scatter(x=timesteps, y=25 * derating, name='Min (Segment 1)', line=dict(dash='dash', color='blue')))
fig.add_trace(go.Scatter(x=timesteps, y=50 * derating, name='Seg 1/2 boundary', line=dict(color='green')))
fig.add_trace(go.Scatter(x=timesteps, y=75 * derating, name='Seg 2/3 boundary', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=timesteps, y=elec_max_derated, name='Max (Segment 3)', line=dict(dash='dash', color='red')))

# Add demand for reference (scaled to fit within operating range)
scaled_demand_preview = 50 + 20 * np.sin(hours * np.pi / 12)
fig.add_trace(go.Scatter(x=timesteps, y=scaled_demand_preview, name='Demand', line=dict(color='black', width=3)))

fig.update_layout(
    title='Time-Varying Piecewise Boundaries (Electricity Output)',
    xaxis_title='Time',
    yaxis_title='Power [kW]',
    legend=dict(x=1.02, y=1),
)
fig.show()

### Build and Solve Time-Varying Model

In [ ]:
fs_time_varying = fx.FlowSystem(timesteps)

# Scale demand to stay within valid operating range (30-70 kW)
# This ensures demand is always above the minimum piecewise boundary
scaled_demand = 50 + 20 * np.sin(hours * np.pi / 12)

fs_time_varying.add_elements(
    # Buses
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    # Cost tracking
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    # Gas supply
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    # Gas engine with TIME-VARYING piecewise efficiency
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas')],
        outputs=[fx.Flow('Elec', bus='Electricity')],
        piecewise_conversion=piecewise_time_varying,  # <-- Time-varying boundaries!
    ),
    # Electricity demand (within valid operating range)
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=scaled_demand)]),
)

fs_time_varying.optimize(fx.solvers.HighsSolver())

In [ ]:
# Plot results with time-varying boundaries using xarray for alignment
results = xr.Dataset(
    {
        'elec_output': fs_time_varying.solution['GasEngine(Elec)|flow_rate'],
        'min_capacity': xr.DataArray(25 * derating, dims=['time'], coords={'time': timesteps}),
        'max_capacity': xr.DataArray(elec_max_derated, dims=['time'], coords={'time': timesteps}),
    }
)

fig = go.Figure()

# Shaded region for valid operating range
fig.add_trace(
    go.Scatter(
        x=list(results['time'].values) + list(results['time'].values)[::-1],
        y=list(results['min_capacity'].values) + list(results['max_capacity'].values)[::-1],
        fill='toself',
        fillcolor='rgba(0,100,255,0.2)',
        line=dict(width=0),
        name='Valid operating range',
    )
)

# Actual operation
fig.add_trace(
    go.Scatter(
        x=results['time'].values, y=results['elec_output'].values, name='Actual output', line=dict(color='red', width=2)
    )
)

# Maximum capacity line
fig.add_trace(
    go.Scatter(
        x=results['time'].values,
        y=results['max_capacity'].values,
        name='Max capacity (derated)',
        line=dict(dash='dash', color='gray'),
    )
)

fig.update_layout(
    title='Engine Operation with Temperature Derating',
    xaxis_title='Time',
    yaxis_title='Electrical Output [kW]',
)
fig.show()

In [ ]:
# Compare all three models
print('Cost Comparison:')
print(f'  Simple (constant efficiency):     {fs_simple.solution["costs"].item():.2f} €')
print(f'  Piecewise (load-varying):         {fs_piecewise.solution["costs"].item():.2f} €')
print(f'  Time-varying piecewise:           {fs_time_varying.solution["costs"].item():.2f} €')

---

## Key Concepts

### PiecewiseConversion Syntax

Define corresponding segments for each flow involved in the conversion:

```python
fx.PiecewiseConversion({
    'Input_Flow': fx.Piecewise([
        fx.Piece(start=input_min_1, end=input_max_1),  # Segment 1
        fx.Piece(start=input_min_2, end=input_max_2),  # Segment 2
    ]),
    'Output_Flow': fx.Piecewise([
        fx.Piece(start=output_min_1, end=output_max_1),  # Segment 1
        fx.Piece(start=output_min_2, end=output_max_2),  # Segment 2
    ]),
})
```

### Time-Varying Boundaries

Pass **arrays** instead of scalars to `Piece` for time-dependent limits:

```python
# Capacity varies with temperature
derating = calculate_derating(temperature_profile)

fx.Piece(
    start=min_capacity * derating,  # Array!
    end=max_capacity * derating,    # Array!
)
```

### Segment Matching

- Each flow must have the **same number of segments**
- Segments operate **together**: when in segment 1, ALL flows use their segment 1 values
- Segments typically **connect** (end of segment N = start of segment N+1)

### Efficiency Calculation per Segment

For each segment, efficiency = output_range / input_range:

| Segment | Fuel Range | Elec Range | Avg Efficiency |
|---------|------------|------------|----------------|
| 1 | 78→132 (54 kW) | 25→50 (25 kW) | 46% |
| 2 | 132→179 (47 kW) | 50→75 (25 kW) | 53% |
| 3 | 179→250 (71 kW) | 75→100 (25 kW) | 35% |

*Note: Actual efficiency varies linearly within each segment.*

## Advanced: Multi-Output Conversion

`PiecewiseConversion` can handle multiple inputs and outputs. For a CHP (Combined Heat and Power):

In [ ]:
# Example: CHP with fuel input, electricity and heat outputs
chp_piecewise = fx.PiecewiseConversion(
    {
        'Fuel': fx.Piecewise(
            [
                fx.Piece(start=100, end=200),
                fx.Piece(start=200, end=300),
            ]
        ),
        'Elec': fx.Piecewise(
            [
                fx.Piece(start=30, end=70),  # 30-35% electrical efficiency
                fx.Piece(start=70, end=100),  # 35-33% electrical efficiency
            ]
        ),
        'Heat': fx.Piecewise(
            [
                fx.Piece(start=45, end=100),  # 45-50% thermal efficiency
                fx.Piece(start=100, end=140),  # 50-47% thermal efficiency
            ]
        ),
    }
)

print('CHP efficiency curve defined with 3 flows and 2 segments')

## Summary

You learned how to:

- Define **piecewise linear efficiency curves** using `PiecewiseConversion`
- Model **load-dependent efficiency** for realistic equipment behavior
- Use **time-varying boundaries** for temperature or condition-dependent limits
- **Visualize** operating ranges and actual operation
- Handle **multi-output conversions** (like CHP)

### When to Use This vs Other Approaches

| Approach | Use When | Example |
|----------|----------|--------|
| Time-varying factors | Efficiency varies with external conditions | Heat pump COP vs temperature |
| **PiecewiseConversion** (this notebook) | Efficiency varies with load level | Gas engine efficiency curve |
| **Time-varying Piecewise** | Operating limits change with conditions | Derating due to temperature |
| PiecewiseEffects | Costs vary non-linearly with size | Economies of scale |

### Next Steps

- **[06c-piecewise-effects](06c-piecewise-effects.ipynb)**: Non-linear investment costs
- **[07-scenarios-and-periods](07-scenarios-and-periods.ipynb)**: Multi-scenario planning